# Data Analysis Pipeline Exercise

**Sequential Flow**: Collect Agent → Analyze Agent → Visualize Agent

This exercise demonstrates:
- Three specialized agents for data workflow
- Sequential data processing pipeline
- Context passing with structured data

In [1]:
from strands import Agent, tool
from strands.models import BedrockModel
import json

## Define Data Collection Tools

In [2]:
@tool
def fetch_sales_data(period: str) -> str:
    """Fetch sales data for specified period"""
    data = {
        "period": period,
        "total_sales": 125000,
        "transactions": 450,
        "avg_order_value": 278,
        "top_products": ["Product A", "Product B", "Product C"],
        "revenue_by_region": {"North": 45000, "South": 35000, "East": 25000, "West": 20000}
    }
    return json.dumps(data)

@tool
def fetch_customer_data(period: str) -> str:
    """Fetch customer metrics for specified period"""
    data = {
        "period": period,
        "new_customers": 85,
        "returning_customers": 365,
        "churn_rate": 12.5,
        "customer_satisfaction": 4.2,
        "nps_score": 42
    }
    return json.dumps(data)

@tool
def fetch_marketing_data(period: str) -> str:
    """Fetch marketing metrics for specified period"""
    data = {
        "period": period,
        "ad_spend": 15000,
        "impressions": 250000,
        "clicks": 8500,
        "conversions": 340,
        "ctr": 3.4,
        "conversion_rate": 4.0
    }
    return json.dumps(data)

## Step 1: Create Collect Agent

**Purpose**: Gather data from multiple sources and consolidate

In [3]:
collect_agent = Agent(
    name="collect_agent",
    system_prompt="""You are a data collection specialist.
    
    Your role:
    - Fetch data from all available sources using the provided tools
    - Consolidate data into a structured format
    - Ensure all data is collected for the requested period
    - Output data in clear JSON format with all sources labeled
    
    Always fetch sales, customer, and marketing data.
    Present the consolidated data clearly for the analysis team.""",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0"),
    tools=[fetch_sales_data, fetch_customer_data, fetch_marketing_data]
)

## Step 2: Create Analyze Agent

**Purpose**: Identify patterns, trends, and insights from collected data

In [4]:
analyze_agent = Agent(
    name="analyze_agent",
    system_prompt="""You are a data analyst specializing in business intelligence.
    
    Your role:
    - Analyze the collected data to identify patterns and trends
    - Calculate key performance indicators (KPIs)
    - Identify correlations between different metrics
    - Highlight strengths, weaknesses, and opportunities
    - Provide actionable insights
    
    Output format:
    ## Key Findings
    - [Finding 1]
    - [Finding 2]
    
    ## Calculated KPIs
    - [KPI 1]: [value]
    - [KPI 2]: [value]
    
    ## Insights & Recommendations
    - [Insight 1]
    - [Insight 2]""",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0")
)

## Step 3: Create Visualize Agent

**Purpose**: Create executive summary with key metrics and visual descriptions

In [5]:
visualize_agent = Agent(
    name="visualize_agent",
    system_prompt="""You are a data visualization specialist creating executive summaries.
    
    Your role:
    - Create a concise executive summary dashboard
    - Highlight top 5 key metrics with clear formatting
    - Describe recommended visualizations (charts/graphs)
    - Use markdown formatting for clarity
    - Include traffic light indicators (🟢 Good, 🟡 Warning, 🔴 Critical)
    
    Output format:
    # Executive Dashboard
    
    ## Key Metrics at a Glance
    | Metric | Value | Status |
    |--------|-------|--------|
    
    ## Recommended Visualizations
    1. [Chart type]: [What to show]
    
    ## Executive Summary
    [3-4 sentence overview]""",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0")
)

## Step 4: Implement Sequential Workflow

In [6]:
def data_analysis_pipeline(query: str) -> dict:
    """
    Execute sequential data analysis pipeline.
    
    Args:
        query: Analysis request with period/scope
    
    Returns:
        Dictionary with outputs from each stage
    """
    print("=" * 60)
    print("STAGE 1: DATA COLLECTION")
    print("=" * 60)
    
    # Stage 1: Collect data from all sources
    collected_data = collect_agent(query)
    print(collected_data)
    
    print("\n" + "=" * 60)
    print("STAGE 2: DATA ANALYSIS")
    print("=" * 60)
    
    # Stage 2: Analyze collected data
    analysis_prompt = f"""Analyze this collected data and provide insights:
    
{collected_data}
"""
    analysis_output = analyze_agent(analysis_prompt)
    print(analysis_output)
    
    print("\n" + "=" * 60)
    print("STAGE 3: VISUALIZATION & SUMMARY")
    print("=" * 60)
    
    # Stage 3: Create visual summary
    viz_prompt = f"""Create an executive dashboard summary from this analysis:
    
DATA:
{collected_data}

ANALYSIS:
{analysis_output}
"""
    final_output = visualize_agent(viz_prompt)
    print(final_output)
    
    return {
        "collected_data": str(collected_data),
        "analysis": str(analysis_output),
        "dashboard": str(final_output)
    }

## Step 5: Test with Realistic Input

In [7]:
# Test Case 1: Monthly business review
query = "Analyze business performance for Q4 2025"

result = data_analysis_pipeline(query)

STAGE 1: DATA COLLECTION
<thinking>To analyze the business performance for Q4 2025, I need to gather sales, customer, and marketing data for the specified period. I will use the provided tools to fetch this data. The period for all the data requests will be "Q4 2025". I will start by fetching the sales data, then the customer data, and finally the marketing data.</thinking>


Tool #1: fetch_sales_data

Tool #2: fetch_customer_data

Tool #3: fetch_marketing_data
<thinking>I have successfully fetched the sales, customer, and marketing data for Q4 2025 from the provided tools. Now, I need to consolidate this data into a structured format in JSON format and label all sources to present it clearly for the analysis team.</thinking>

<thinking>The JSON format will have three main sections: sales, customer, and marketing, each with their respective data points. I will make sure to include the period and source label for each data point to ensure clarity.</thinking>

Here is the consolidated da

## Step 6: Test with Different Query

In [8]:
# Test Case 2: Marketing campaign analysis
query = "Evaluate marketing ROI and customer acquisition for December 2025"

result = data_analysis_pipeline(query)

STAGE 1: DATA COLLECTION
<thinking>To evaluate the marketing ROI and customer acquisition for December 2025, I will need to extract specific metrics from the marketing data fetched earlier. The marketing ROI can be calculated by dividing the net profit from marketing activities by the total ad spend, then multiplying by 100 to get a percentage. Customer acquisition will involve looking at the number of new customers acquired during that month.</thinking>

<thinking>Since the data provided is for Q4 2025, which includes October, November, and December, I will assume that the marketing data represents the total for the quarter. To find the December figures, I will need to estimate based on the quarterly data, assuming a roughly even distribution of activities across the quarter. However, without specific December data, this will be an approximation.</thinking>

<thinking>For customer acquisition, since we only have Q4 data, I will include the total number of new customers for Q4 and note

## Step 7: Inspect Individual Stages

In [9]:
# Access individual stage outputs
print("=== COLLECTED DATA ===")
print(result['collected_data'][:400] + "...\n")

print("=== ANALYSIS ===")
print(result['analysis'][:400] + "...\n")

print("=== DASHBOARD ===")
print(result['dashboard'][:400] + "...")

=== COLLECTED DATA ===
<thinking>To evaluate the marketing ROI and customer acquisition for December 2025, I will need to extract specific metrics from the marketing data fetched earlier. The marketing ROI can be calculated by dividing the net profit from marketing activities by the total ad spend, then multiplying by 100 to get a percentage. Customer acquisition will involve looking at the number of new customers acqui...

=== ANALYSIS ===
## Key Findings
- **Marketing ROI**: Based on the assumption of an even distribution of marketing activities across Q4 2025, the estimated ROI for December 2025 is derived from the quarterly data. However, without specific December data, the exact ROI for December cannot be precisely determined.
- **Customer Acquisition**: There were 85 new customers acquired in Q4 2025. Assuming an even distribut...

=== DASHBOARD ===
# Executive Dashboard

## Key Metrics at a Glance
| Metric | Value | Status |
|--------|-------|--------|
| Estimated Marketing ROI 

## Exercise Variations

Try modifying the pipeline:

1. **Add Data Validation Agent** after Collection
2. **Add Anomaly Detection Agent** in Analysis stage
3. **Add Forecasting Agent** after Analysis
4. **Implement parallel collection** from multiple data sources
5. **Add feedback loop** for data quality issues

## Key Learnings

✅ **Sequential Flow**: Collect → Analyze → Visualize  
✅ **Tool Integration**: Collect agent uses multiple data source tools  
✅ **Context Passing**: Full data passed through pipeline  
✅ **Specialized Roles**: Each agent has distinct analytical purpose  
✅ **Structured Output**: JSON data → Insights → Executive summary  